# SVM classification on the Ozone dataset
Author: Tomas Hobza

In [2]:
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from sklearn.svm import LinearSVC
import time
from sklearn.metrics import balanced_accuracy_score

# set random seed for reproducibility
np.random.seed(0)

## 0. Load the dataset

In [ ]:
# Load the parquet files
df = pd.read_parquet('br_train_cleaned.parquet')


# split the data into features and labels
X = df.drop(columns=['class'])
y = df['class']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

class
False    177
True     108
Name: count, dtype: int64

1. Experiment 